![](ozet.JPG)

# Basit Ozetleme

In [48]:
c1 = ['w1', 'w2', 'w3']
c2 = ['w2', 'w4', 'w5']
c3 = ['w1', 'w6', 'w7']
c4 = ['w2', 'w4', 'w8']

sentences = [c1, c2, c3 , c4]
sentences

[['w1', 'w2', 'w3'],
 ['w2', 'w4', 'w5'],
 ['w1', 'w6', 'w7'],
 ['w2', 'w4', 'w8']]

In [49]:
words = set(c1 + c2 + c3 + c4)
words

{'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8'}

In [50]:
scores = dict()
for w in words:
    scores[w] = 0
    for c in sentences:
        if w in c:
            scores[w] += 1
scores

{'w6': 1, 'w1': 2, 'w7': 1, 'w4': 2, 'w3': 1, 'w8': 1, 'w2': 3, 'w5': 1}

In [55]:
sentence_scores = [0] * len(sentences)
for i, c in enumerate(sentences):
    sentence_score = 0
    for w in c:
        sentence_score += scores[w]
    sentence_scores[i] = sentence_score
sentence_scores

[6, 6, 4, 6]

In [58]:
scores = np.array(sentence_scores)
select = scores.argsort()
select

array([2, 0, 1, 3])

In [59]:
# Generate summary
for i in select[-3:]:
    print(sentences[i])

['w1', 'w2', 'w3']
['w2', 'w4', 'w5']
['w2', 'w4', 'w8']


# Gelişmiş Özetleme

In [1]:
import re
import time
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity

word2vec Turkish: https://github.com/akoksal/Turkish-Word2Vec

In [2]:
start = time.time()

path = '/Users/uzaycetin/Desktop/INF236-word2vec/'
word_vectors = KeyedVectors.load_word2vec_format(path + 'trmodel', binary=True)

print("total time: ", time.time() - start)

total time:  4.114732027053833



Turkce stopwords lerden kurtul : https://github.com/sgsinclair/trombone/blob/master/src/main/resources/org/voyanttools/trombone/keywords/stop.tr.turkish-lucene.txt

Kelime kokune in

In [3]:
url ='https://raw.githubusercontent.com/sgsinclair/trombone/master/src/main/resources/org/voyanttools/trombone/keywords/stop.tr.turkish-lucene.txt'
data = pd.read_csv(url)

In [4]:
data

,# Turkish stopwords from LUCENE-559
0,"# merged with the list from ""Information Retri..."
1,# (http://www.users.muohio.edu/canf/papers/J...
2,acaba
3,altmış
4,altı
...,...
206,yine
207,yirmi
208,yoksa
209,yüz


In [5]:
stpwrds = set(data[2:].iloc[:, 0].values)

In [111]:
def vectorize(cumle, stpwrds = stpwrds):
    kelimeler = cumle.split()
    vc = np.zeros((1, 400))
    n = 0
    for v in kelimeler:
        if v in stpwrds:
            continue
        try:
            vc += word_vectors[v].reshape(1,400)
            n +=1
        except:
            continue
    vc /= n 
    return vc

# Ozetleme

In [112]:
sentences = ['Fenerbahçe spor klubü ciddi başarılar kazandı sen ben ben',
             'Damlayan su, taşı deler ben sen o. Ovidius',
             'Hazine, eziyet çekene gözükür ben sen ben',
             'Fenerbahçe vurdu gol oldu',
             'Galatasaray voleybol takımı kazanır']

clean_sentences = [re.sub('[;!@#’‘?.,\'$]', '', s).lower() for s in sentences]
clean_sentences

['fenerbahçe spor klubü ciddi başarılar kazandı sen ben ben',
 'damlayan su taşı deler ben sen o ovidius',
 'hazine eziyet çekene gözükür ben sen ben',
 'fenerbahçe vurdu gol oldu',
 'galatasaray voleybol takımı kazanır']

In [113]:
vectors = [vectorize(s) for s in clean_sentences]

In [114]:
n = len(sentences)
X = np.concatenate(vectors).reshape(n,1,400)
X.shape

(5, 1, 400)

# Cosine

In [115]:
def dot(X, Y):
    result = 0
    for i in range(len(X)):
        result += X[i] * Y[i]
    return result

def norm(X):
    result = dot(X, X)
    return result**0.5

def cos(X, Y):
    return dot(X, X)/ (norm(X) * norm(Y))

In [116]:
cos(X[0].flatten(), X[0].flatten())

1.0000000000000002

In [117]:
cosine_similarity(X[0], X[0])

array([[1.]])

In [118]:
sim_mat = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        sim_mat[i,j] = cos(X[i].flatten(), X[j].flatten())

In [119]:
sim_mat

array([[1.        , 1.41828768, 1.93636713, 0.67920516, 0.71931469],
       [0.70507558, 1.        , 1.36528517, 0.47889097, 0.50717122],
       [0.51643099, 0.73244771, 1.        , 0.35076259, 0.3714764 ],
       [1.47230919, 2.08815798, 2.85093113, 1.        , 1.05905363],
       [1.39021212, 1.97172072, 2.69196106, 0.94423925, 1.        ]])

In [120]:
scores = sim_mat.sum(axis = 1)
scores

array([5.75317466, 4.05642293, 2.9711177 , 8.47045193, 7.99813314])

In [121]:
select = scores.argsort()
select

array([2, 1, 0, 4, 3])

In [122]:
select[-3:]

array([0, 4, 3])

In [123]:
# Generate summary
for i in select[-3:]:
    print(sentences[i])

Fenerbahçe spor klubü ciddi başarılar kazandı sen ben ben
Galatasaray voleybol takımı kazanır
Fenerbahçe vurdu gol oldu


Stop words atılmadıgında yanlıs ozet

```
Hazine, eziyet çekene gözükür ben sen ben
Galatasaray voleybol takımı kazanır
Fenerbahçe vurdu gol oldu
```